<div align="right" dir="rtl">
<h1> Recommender System </h1>
در این تمرین قصد داشتیم که یک سرویس توصیه کننده بازی کاربر به کاربر را به کمک اسپارک پیاده سازی کنیم که رویکرد مورد استفاده ما در این پیاده سازی، رویکرد collabrative filtering بوده است زیرا در این جا اطلاعات خوبی از کاربران داریم. خود این روش نیز به دو دسته تقسیم میگردد: user based & item based که در اینجا روش مبتنی بر کاربر مدنظر ما بوده است
</div>




<div align="right" dir="rtl">
در ابتدا لازم بود که کتابخانه pyspark را به پروژه اضافه کنیم تا از آن جهت پیاده¬سازی اسپارک استفاده کنیم
</div>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e55017492f1ebb2c0bb5e0ad7d211197088ea5347e7b355461a509764e7e8be1
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark



<div align="right" dir="rtl">
در مرحله بعد نیاز به یک evaluator نیاز داشتیم که از رگرسیون خود کتابخانه pyspark  استفاده نمودیم
</div>


In [2]:
from pyspark.ml.evaluation import RegressionEvaluator

<div align="right" dir="rtl">
اما برای هندل کردن بحث recommendation از ALS استفاد نمودیم. ALS خودش مخفف Alternative Least Square	 است که ما از آن جهت محاسبه میزان مشابهت انجام توصیه استفاده می¬کنیم. ALS ابزارهای مختلفی در اختیار ما می¬گذارد که کار انجام توصیه و ساخت ابزار توصیه گر را آسان می¬سازد
</div>


In [3]:
from pyspark.ml.recommendation import ALS


<div align="right" dir="rtl">
در مرحله بعد لازم است که یک سشن اسپارک جهت استفاده از اسپارک ایجاد کنیم
</div>
 


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName("userToUserRecommender").getOrCreate()


<div align="right" dir="rtl">
حال باید فایل¬های مربوط به دیتاست بازی¬ها و نمرات آن¬ها را توسط اسپارک بخوانیم تا RDD از آن¬ها ساخته بشود. header = true برای این است که سطر اول به عنوان هدر ست گردد و inferschema = True نیز برای آن است که خود اسپارک به شکل اتومات تایپ داده¬ها را متوجه گردد.
</div>


In [6]:
ratings_df = spark.read.csv("ratings.csv", inferSchema= True, header = True)

In [7]:
games_df = spark.read.csv("games.csv", inferSchema= True, header = True)


<div align="right" dir="rtl">
حال میخواهیم که نگاهی به دیتاست¬هایمان بیاندازیم درنتیجه 10 تا رکورد از هرکدام را نمایش می¬دهیم. همچنین دستور دیسکرایب نیز یک خلاصه از دیتاستمان شامل میانگین، تعداد ، انحراف معیار و حداقل و حداکثر داده¬ها به ما می¬دهد
</div>
 


In [8]:
ratings_df.show(10)
ratings_df.describe().show()

+-------+-------+------+
|game_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
+-------+-------+------+
only showing top 10 rows

+-------+-----------------+------------------+------------------+
|summary|          game_id|           user_id|            rating|
+-------+-----------------+------------------+------------------+
|  count|           981548|            981548|            981548|
|   mean|4943.316270829343|25616.592174809586|3.8564950465998606|
| stddev|2873.219878464268|15228.359436640818|0.9839536925631116|
|    min|                1|                 1|                 1|
|    max|            10000|             53424|                 5|
+-------+-----------------+------------------+------------------+



In [9]:
games_df.show(10)
games_df.describe().show()

+-------+--------------------+------------+--------------------+--------------------+
|game_id|                name|release_date|             summary|          meta_score|
+-------+--------------------+------------+--------------------+--------------------+
|      1|The Legend of Zel...|   23-Nov-98|As a young boy, L...|                  99|
|      2|Tony Hawk's Pro S...|   20-Sep-00|As most major pub...|                  98|
|      3| Grand Theft Auto IV|   29-Apr-08|"[Metacritic's 20...| fresh off the bo...|
|      4|         SoulCalibur|    8-Sep-99|This is a tale of...|                  98|
|      5|  Super Mario Galaxy|   12-Nov-07|[Metacritic's 200...|                  97|
|      6|Super Mario Galaxy 2|   23-May-10|Super Mario Galax...|                  97|
|      7|Red Dead Redempti...|   26-Oct-18|Developed by the ...|                  97|
|      8|  Grand Theft Auto V|   18-Nov-14|Grand Theft Auto ...|                  97|
|      9|Disco Elysium: Th...|   30-Mar-21|Disco Elysi


<div align="right" dir="rtl">
حال برای اینکه بتوانیم برروی دیتاست¬هایمان که در اسپارک قرار دادیم، به راحتی بتوانیم کوئری بزنیم، از آن¬ها یک  Table ایجاد می¬کنیم تا در ادامه بتوانیم از آن¬ها به شکل مناسب استفاده کنیم
</div>


In [10]:
ratings_df.registerTempTable("Ratings")
games_df.registerTempTable("Games")

/usr/local/lib/python3.8/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


<div align="right" dir="rtl">
در این تسک توصیه کننده ما، دیتاست اصلی مورد استفاده ما، دیتاست ratingها هست و دیتاست بازی¬ها صرفا اطلاعات بیشتری از بازی را به ما می¬دهد در نتیجه آن¬ها را جوین می¬کنیم و در قالب یک RDD نگه می¬داریم. عملیات جوین براساس آیدی بازی¬ها صورت می¬گیرد
همچنین نمونه نرمالایز شده¬ای از آن را  نیز نگه می¬داریم که به روش min-max آن را نرمالایز کرده¬ایم که فرمول آن به صورت زیر است.
<br>
X – Xmin / Xmax - Xmin
</div>




In [11]:
aggregate_data = spark.sql("SELECT Games.game_id, Ratings.user_id, Ratings.rating, Games.name, Games.release_date, Games.summary, Games.meta_score FROM Ratings INNER JOIN Games On Ratings.game_id = Games.game_id")
aggregate_data.show()
aggregate_data_normal = spark.sql("SELECT Games.game_id, Ratings.user_id, Ratings.rating / 5 AS rating , Games.name, Games.release_date, Games.summary, Games.meta_score FROM Ratings INNER JOIN Games On Ratings.game_id = Games.game_id")
aggregate_data_normal.show()


+-------+-------+------+--------------------+------------+--------------------+----------+
|game_id|user_id|rating|                name|release_date|             summary|meta_score|
+-------+-------+------+--------------------+------------+--------------------+----------+
|      1|    314|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|    439|     3|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|    588|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   1169|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   1185|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2077|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2487|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2900|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|


<div align="right" dir="rtl">
حال لازم است که دیتاستمان را به دو بخش تست و آموزش تقسیم کنیم. برای این کار 80 درصد از داده¬ها را برای آموزش و 20 درصد را برای تست قرار دادیم که طبق بررسی¬هایی که انجام دادیم نتایج مطلوبی را به ما ارائه می¬داد
</div>


In [12]:
(train_dataset, test_dataset) = aggregate_data.randomSplit([0.8, 0.2], seed= 42)


<div align="right" dir="rtl">
حال لازم است که پارامتر¬های لازم برای انجام توصیه توسط ALS را مشخص نماییم که این پارارمتر¬ها شامل تعداد تکرار، پارامتر رگرسیون و تعیین user و آیتم مورد بررسی و معیار rating است
</div>


In [13]:
als_factorization = ALS(maxIter= 10, regParam= 0.1, itemCol= "game_id", userCol="user_id", ratingCol= "rating")


<div align="right" dir="rtl">
حال در رویکرد collaborative filtering ما لازم است که سطر¬هایی که دارای مقادیر خالی هستند (Nan) را دراپ کنیم که با دستور زیر این کار امکان¬پذیر می¬باشد
</div>
 


In [14]:
als_factorization.setColdStartStrategy("drop")

ALS_5c0b2a35e105


<div align="right" dir="rtl">
در این خط نیز مدلمان را براساس داده¬های آموزش فیت می¬کنیم.
</div>


In [15]:
predictor_model = als_factorization.fit(train_dataset)


<div align="right" dir="rtl">
حال بعد از فیت شدن مدل لازم است که یک ترنسفورم برروی داده¬های تستمان براساس داده¬های آموزشی¬مان داشته باشیم
</div>
 


In [16]:
predictor = predictor_model.transform(test_dataset)


<div align="right" dir="rtl">
حال میخواهیم نتایج پیش¬بینی شده را مشاهده کنیم 
</div>
 


In [17]:
predictor.show()

+-------+-------+------+--------------------+------------+--------------------+--------------------+----------+
|game_id|user_id|rating|                name|release_date|             summary|          meta_score|prediction|
+-------+-------+------+--------------------+------------+--------------------+--------------------+----------+
|      3|  32592|     5| Grand Theft Auto IV|   29-Apr-08|"[Metacritic's 20...| fresh off the bo...| 2.9819381|
|      7|  19984|     5|Red Dead Redempti...|   26-Oct-18|Developed by the ...|                  97|  4.704021|
|     20|  32592|     4|            BioShock|   21-Aug-07|"[Metacritic's 20...|"" ""monster-clos...|  3.737324|
|     22|  19984|     5|Uncharted 2: Amon...|   13-Oct-09|Fortune hunter Na...|                  96|  4.934289|
|     26|  35982|     2|            Tekken 3|   29-Apr-98|An ancient evil f...|                  96| 2.2924564|
|     27|  32592|     4|       Mass Effect 2|   26-Jan-10|The Mass Effect t...|                  96| 4.3


<div align="right" dir="rtl">
حال لازم است که میزان خطای توصیه¬گرمان در مقادیر پیش¬بینی شده را محاسبه کنیم. برای این کار از معیار خطای RMSE استفاده نمودیم که مخفف Root Mean Square Error می¬باشد
</div>
 


In [19]:
evaluator_model = RegressionEvaluator(metricName= "rmse", labelCol="rating", predictionCol="prediction")

In [20]:
rmse = evaluator_model.evaluate(predictor)
print(f"RMSE Error rate: {rmse}")

RMSE Error rate: 0.9020742770912595



<div align="right" dir="rtl">
حال در این بخش تابعی طراحی کردیم که آیدی کاربر را می¬گیرد و ریتینگ¬های پیش¬بینی شده توسط آن کاربر برای بازی¬ها را به شکل یک RDD به ما برمی¬گرداند
</div>



In [21]:
def userFilter (test, id):
  return test.filter(test['user_id'] == str(id)).select(["game_id", 'user_id','name'])


specific_user = userFilter(test_dataset, 314)
specific_user.show()


+-------+-------+--------------------+
|game_id|user_id|                name|
+-------+-------+--------------------+
|     17|    314|The House in Fata...|
|     35|    314|Baldur's Gate II:...|
|     51|    314|Metal Gear Solid ...|
|     56|    314|          God of War|
|     66|    314|The Elder Scrolls...|
|     84|    314|Sid Meier's Civil...|
|    101|    314|Ori and the Will ...|
|    104|    314|Super Mario 3D World|
|    125|    314|          Beat Saber|
|    140|    314|World Soccer Winn...|
|    159|    314|Super Street Figh...|
|    183|    314|Unreal Tournament...|
|    185|    314|        Wave Race 64|
|    239|    314|     Forza Horizon 3|
|    267|    314|God of War Collec...|
|    327|    314|            NHL 2001|
|    342|    314| Panzer Dragoon Orta|
|    352|    314|WWF SmackDown! 2:...|
|    525|    314|  Railroad Tycoon II|
|    923|    314|Dragon Quest Buil...|
+-------+-------+--------------------+
only showing top 20 rows




<div align="right" dir="rtl">
در نهایت برای یکی از کاربران که آیدی آن 314 است اینکار را انجام می¬دهیم و نتایج پیش¬بینی شده را مرتب می¬کنیم تا لیست بازی¬هایی که کاربر امکان دارد بیشترین امتیاز را بدهد به دست می¬آوریم تا از آن جهت توصیه کردن استفاده نماییم
</div>



In [22]:
specific_user = userFilter(test_dataset, 314)
result_prediction = predictor_model.transform(specific_user)
result_prediction.orderBy("prediction", ascending = False).show()

+-------+-------+--------------------+----------+
|game_id|user_id|                name|prediction|
+-------+-------+--------------------+----------+
|    267|    314|God of War Collec...| 4.1655264|
|    140|    314|World Soccer Winn...| 3.9897256|
|   3712|    314|Fluidity: Spin Cycle| 3.8177567|
|   4720|    314|Naruto Shippuden:...| 3.8161302|
|     17|    314|The House in Fata...| 3.7494202|
|   1838|    314|           FIFA 2001|  3.724308|
|    185|    314|        Wave Race 64| 3.6927977|
|    352|    314|WWF SmackDown! 2:...|  3.659255|
|    327|    314|            NHL 2001| 3.6468258|
|    101|    314|Ori and the Will ...|  3.631939|
|   3355|    314|               Black|  3.629798|
|    125|    314|          Beat Saber| 3.6193683|
|     66|    314|The Elder Scrolls...| 3.6066704|
|    159|    314|Super Street Figh...| 3.5979755|
|    923|    314|Dragon Quest Buil...| 3.5905871|
|    183|    314|Unreal Tournament...| 3.5476847|
|    239|    314|     Forza Horizon 3|  3.538364|
